# 制約とペナルティ

## 制約付き最適化問題

数理最適化において、制約とは解が満たさなければならない条件のことです。例えば、次の問題は制約付き最適化問題になります。

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) = 0
\end{aligned}
$$

ここで$f$と$g$は決定変数$x$の関数です。条件$g(x) = 0$は等式制約と呼ばれます。$g(x) = 0$を満たすすべての$x$の集合は実行可能集合と呼ばれます。制約は$g(x) = 0$のような等式制約に限らず、$g(x) \leq 0$のような不等式制約である場合もあります。例えば、次の問題も制約付き最適化問題です。

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) \leq 0
\end{aligned}
$$

`jijmodeling`では、`Constraint`クラスを使用して等式制約と不等式制約の両方を記述できます。例えば、等式制約$\sum_i x_i = 1$は次のように表現できます。

In [1]:
import jijmodeling as jm

N = jm.Placeholder("N")
x = jm.BinaryVar("x", shape=(N,))
n = jm.Element('n', belong_to=(0, N))

jm.Constraint("onehot", jm.sum(n, x[n]) == 1)

Constraint(name="onehot", expression=sum(n in [0..N), x[n]) == 1)

上記のコードでは、`jm.Constraint`の第1引数として文字列"onehot"を指定していることに注意してください。制約オブジェクトには名前と制約式があります。これらの名前は制約が満たされているかどうかを確認する際に使用されます。制約式は、`==`、`<=`、または`>=`の3つの比較演算子のいずれかを使用する論理式でなければなりません。また、次のようにして1つの問題に複数の制約を課すことができます。

In [2]:
x = jm.BinaryVar("x", shape=(4,))
problem = jm.Problem("constraint_sample")
problem += jm.Constraint("c01", x[0] + x[1] <= 1)
problem += jm.Constraint("c12", x[1] + x[2] <= 1)
problem += jm.Constraint("c23", x[2] + x[3] >= 0)

:::{tip}

他の比較演算子（例えば`>`）や論理演算子はサポートされていません。

```python
x = jm.BinaryVar("x", shape=(4,))
jm.Constraint("unsupported", (x[0] + x[1] <= 1) | (x[1] + x[2] <= 1))
```

:::

### `forall`制約
しばしば制約は変数によってインデックス付けされます。例えば、次のような問題は制約付き最適化問題です。

$$
\begin{aligned}
\text{Minimize} & \quad \sum_{i=0}^{N-1} \sum_{j=0}^{M-1} a_{ij} x_{ij} \\
\text{subject to} & \quad \sum_{j = 0}^{M - 1} x_{ij} = 1 \quad \forall i \in \left\{0, \ldots, N - 1\right\}
\end{aligned}
$$

このような$\forall i \in \left\{0, \ldots, N - 1\right\}$を表現するために、`Constraint`オブジェクトには`forall`オプションがあります。例えば、上記の問題は次のように表現できます。

In [ ]:
N = jm.Placeholder("N")
M = jm.Placeholder("M")
a = jm.Placeholder("a", ndim=2)
x = jm.BinaryVar("x", shape=(N, M))
i = jm.Element('i', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))

problem = jm.Problem("forall_sample")
problem += jm.sum([i, j], a[i, j] * x[i, j])
problem += jm.Constraint("onehot", jm.sum(j, x[i, j]) == 1, forall=i)

## ペナルティとは

[ペナルティ法](https://en.wikipedia.org/wiki/Penalty_method)と[ラグランジュ乗数法](https://en.wikipedia.org/wiki/Lagrange_multiplier) は制約付き最適化問題を制約なし最適化問題に変換するための最も一般的な方法です。ここではペナルティ法について見ていきましょう。

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) = 0
\end{aligned}
$$

この問題は次のような制約なし最適化問題に変換されます。

$$
\text{Minimize} \quad f(x) + \alpha p(x),
$$

この変換では$\alpha$（ペナルティ係数またはラグランジュ乗数）と$p(x)$（ペナルティ項）が重要な役割を果たします。通常、$p(x)$は$p(x) = g(x)^2$として定義されます。$f(x) + \alpha p(x)$の最小値が$p(x) = 0$を満たす場合、その$x$は元の制約付き最適化問題の最小値になります。ペナルティ$p(x)$が正の場合、ペナルティ係数$\alpha$の値を増やして上記の制約なし最適化問題を解くと元の最適化問題の解を得られる可能性が高まります。

一部のソルバーは制約なし最適化問題のみしか受け入れません。QUBOの「U」は「Unconstrained（無制約）」を意味しています。`jijmodeling`で制約付き最適化問題として定式化した数理モデルを入力形式がQUBOのソルバーに与えるためには、`jijmodeling_transpiler`またはJijZeptを利用して制約なし最適化問題に変換する必要があります。しかし、実際の最適化問題においては$p$の定義も重要であり、そのため、`jijmodeling`ではペナルティ項をカスタマイズする機能を提供しています。

### 制約をペナルティ項に変換する

`jijmodeling`は制約をペナルティ項に変換する機能を持ちません。ここでは`jijmodeling_transpiler`が制約をペナルティ項に変換する方法を説明します。以下のような簡単な問題を考えてみましょう。

$$
\begin{aligned}
\text{Minimize} & \quad \sum_{i=0}^{N-1} a_i x_i \\
\text{subject to} & \quad \sum_{i = 0}^{N - 1} x_i = 1
\end{aligned}
$$

この問題は`jijmodeling`で次のように定式化できます。

In [4]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=1)
N = a.len_at(0, latex="N")

i = jm.Element('i', belong_to=(0, N))
x = jm.BinaryVar('x', shape=(N,))

problem = jm.Problem('translate_constraint_to_penalty')
problem += jm.sum(i, a[i]*x[i])
problem += jm.Constraint(
    'onehot',
    jm.sum(i, x[i]) == 1,
)
problem

`jijmodeling_transpiler`では、この制約付き最適化問題は次の制約なし最適化問題に変換されます。

$$
\text{Minimize} \quad \sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
$$

ここでは、$a = [1, 2]$とし、$\alpha = 5$とする場合を見てみましょう。

In [ ]:
import jijmodeling_transpiler as jmt

instance_data = {
    "a": [1, 2],
}

compiled_model = jmt.core.compile_model(problem, instance_data)
pubo_builder = jmt.core.pubo.transpile_to_pubo(
    compiled_model,
    normalize=False  # 簡単のために正規化を無効化
)
qubo, constant = pubo_builder.get_qubo_dict(multipliers={ 'onehot': 5 })

結果は次のようになります。

In [6]:
qubo

{(0, 0): -4.0, (1, 1): -3.0, (0, 1): 10.0}

In [7]:
constant

5.0

なぜ、このような`qubo`と`constant`が得られるのかというと、以下のような計算が行われたからです。

$$
\begin{aligned}
\sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
&= x_1 + 2 x_2 + 5 (x_1 + x_2 - 1)^2 \\
&= -4 x_1 - 3 x_2 + 10 x_1 x_2 + 5 \\
&= \begin{bmatrix}
    x_1 & x_2 \end{bmatrix}
\begin{bmatrix}
    -4 & 10 \\
     0 & -3
\end{bmatrix}
\begin{bmatrix}
    x_1 \\
    x_2
\end{bmatrix}
+ 5
\end{aligned}
$$

上記の計算は、バイナリ変数$x_i$が$x_i^2 = x_i$であることを使って式変形しています。

`jijmodeling_transpiler`の変換過程には2つのフェーズに分かれています。

- `Problem`オブジェクトと`instance_data`を使用して`CompiledInstance`オブジェクトに変換する。
- `CompiledInstance`オブジェクトに未定乗数を指定してQUBOに変換する。

:::{note}

`jijmodeling_transpiler`では、`get_qubo_dict`メソッドの`multipilers`引数を使用することで各ペナルティ項の未定乗数を設定することができます。また、`jijmodeling_transpiler`では[拡張ラグランジュ法](https://en.wikipedia.org/wiki/Augmented_Lagrangian_method)のような他の緩和方法もサポートされています。詳細については[jijmodeling_transpilerのリファレンス](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/references/jijmodeling_transpiler/core/pubo/)を確認してください。

:::

### `CustomPenaltyTerm`

制約をペナルティ項に変換する作業は`jijmodeling_transpiler`やJijZeptの各種サービスの役割ですが、必要に応じてオリジナルのペナルティ項を設定したい場合も存在します。`jijmodeling`では`CustomPenaltyTerm`を使用することでペナルティ項のカスタマイズが可能になります。ここでは、前述の例と同じペナルティ項を`CustomPenaltyTerm`で定義する方法を説明します。

$$
\text{Minimize} \quad \sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
$$

この問題は次のように`CustomPenaltyTerm`を使用して表現できます。

In [ ]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=1)
N = a.len_at(0, latex="N")

i = jm.Element('i', belong_to=(0, N))
x = jm.BinaryVar('x', shape=(N,))

problem = jm.Problem('penalty_sample')
problem += jm.sum(i, a[i]*x[i])
problem += jm.CustomPenaltyTerm(
    'onehot',
    (jm.sum(i, x[i]) - 1)**2,
)

この時点では未定乗数$\alpha$が設定されていないことに注意してください。この場合においても前述と同じように`jijmodeling_transpiler`を利用することができます。

In [9]:
import jijmodeling_transpiler as jmt

instance_data = {
    "a": [1, 2],
}

compiled_model = jmt.core.compile_model(problem, instance_data)
pubo_builder = jmt.core.pubo.transpile_to_pubo(
    compiled_model,
    normalize=False  # 簡単のために正規化を無効化
)
qubo, constant = pubo_builder.get_qubo_dict(multipliers={ 'onehot': 5 })

当然、結果は同じになります。

In [10]:
qubo

{(0, 0): -4.0, (1, 1): -3.0, (0, 1): 10.0}

In [11]:
constant

5.0

また、`CustomPenaltyTerm`にも`forall`引数があります。

In [12]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=2)
N = a.len_at(0, latex="N")
M = a.len_at(1, latex="M")

i = jm.Element('i', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))
x = jm.BinaryVar('x', shape=(N, M))

problem = jm.Problem('forall_penalty_sample')
problem += jm.sum([i, j], a[i, j]*x[i, j])
problem += jm.CustomPenaltyTerm(
    'onehot',
    (jm.sum(j, x[i, j]) - 1)**2,
    forall=i
)
problem